Evaluating models trained on SGD

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# imports
import re
from copy import deepcopy
from pathlib import Path
from typing import *

import pandas as pd
import srsly
import swifter
import wandb
from datasets import load_from_disk
from hydra.utils import instantiate
from omegaconf import OmegaConf
from tqdm.auto import tqdm

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import ticker
from polyfuzz import PolyFuzz
from polyfuzz.models import TFIDF, EditDistance
from rapidfuzz import fuzz
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from src.data.utilities import (
    check_dict_equal,
    clean_slot_values,
    clean_time,
    complement_labels,
    diff_train,
    extract_domains,
    extract_slots,
    extract_values,
    remove_empty_slots,
)
from src.evaluation import compute_prf, diff, jga, prepare_states_eval, slot_metrics
from src.model import DSTTask

In [3]:
list_dfs = []
path = Path("../preds")

for exp_path in list(path.rglob("*experiment_3")):

    for p in tqdm(list(exp_path.rglob("*preds.parquet")), desc=exp_path.name):

        # load
        df = pd.read_parquet(p).assign(
            split=p.parents[0].name,
            epoch=int(p.parents[1].name.split("=")[1]),
            version=p.parents[2].name,
            model=p.parents[4].name,
            size=p.parents[5].name,
        )

        # extract slots from experiments with normal states
        df = df.assign(
            states=lambda df_: df_["states"].map(remove_empty_slots),
            previous_states=lambda df_: df_["previous_states"].map(remove_empty_slots),
        )

        list_dfs.append(df)

experiment_3:   0%|          | 0/40 [00:00<?, ?it/s]

In [4]:
df = pd.concat(list_dfs).reset_index(drop=True)

In [5]:
df.groupby(["model", "split", "version"])[["epoch", "dialogue_id"]].nunique()

epoch  \
model                                        split      version          
sgd_cum_fullhist+nostate_2022-12-27T15-28-10 test       v1          10   
                                             validation v1          10   
sgd_ops_partialhist+prev_2022-12-27T15-24-56 test       v1          10   
                                             validation v1          10   

                                                                 dialogue_id  
model                                        split      version               
sgd_cum_fullhist+nostate_2022-12-27T15-28-10 test       v1              4201  
                                             validation v1              2482  
sgd_ops_partialhist+prev_2022-12-27T15-24-56 test       v1              4201  
                                             validation v1              2482

In [6]:
df["states"] = df["states"].swifter.apply(prepare_states_eval)

Pandas Apply:   0%|          | 0/1333200 [00:00<?, ?it/s]

In [59]:
# load gold annotations
dataset_dict = load_from_disk(f"../data/processed/schema_guided_dialogue")

true_df = (
    pd.concat([dataset_dict[split].to_pandas().assign(split=split) for split in ("test", "validation")])
    .sort_values(["dialogue_id", "turn_id"])
    .reset_index(drop=True)
    .assign(states=lambda df_: df_[f"states"].map(lambda ex: prepare_states_eval(remove_empty_slots(ex))))
)

print(len(true_df))
true_df = true_df.loc[true_df["usr_utt"] != "none", ["dialogue_id", "turn_id", "states"]]
print(len(true_df))

73343
66660


In [60]:
# add gold annotations
pred_df = pd.merge(df, true_df, on=["dialogue_id", "turn_id"], how="inner", suffixes=["_pred", ""])

In [61]:
# compute correct predictions
pred_df["jga_turn"] = pred_df.swifter.apply(
    lambda row: jga(row["states_pred"], row["states"]),
    axis=1,
)

Pandas Apply:   0%|          | 0/2040000 [00:00<?, ?it/s]

In [10]:
eval_cols = ["jga_turn"]
index_cols = ["model", "version", "split", "size", "epoch"]

In [11]:
results = pred_df.groupby(index_cols)[eval_cols].mean()

In [12]:
validation_results = []
for col in eval_cols:
    val_best_ids = results.groupby(index_cols[:-1])[col].idxmax().reset_index()
    val_best_ids = val_best_ids.loc[val_best_ids["split"] == "validation"].assign(
        best_ckpt=lambda df_: df_[col].map(lambda ex: (ex[0], ex[1], ex[3], ex[4]))
    )
    dd = results.loc[results.index.droplevel("split").isin(val_best_ids["best_ckpt"]), [col]].assign(
        metric_name=col, metric=lambda df_: df_[col] * 100
    )[["metric_name", "metric"]]

    validation_results.append(dd)


validation_results = pd.concat(validation_results).reset_index()
validation_results["exp"] = validation_results["model"].str.split("_").map(lambda ex: f"{ex[1]}_{ex[2]}")
validation_results = (
    validation_results.groupby(index_cols[1:-1] + ["exp", "metric_name"])["metric"].describe().reset_index()
)

In [13]:
validation_results

,version,split,size,exp,metric_name,count,mean,std,min,25%,50%,75%,max
0,v1,test,base,cum_fullhist+nostate,jga_turn,1.0,0.210116,NaN,0.210116,0.210116,0.210116,0.210116,0.210116
1,v1,test,base,ops_partialhist+prev,jga_turn,1.0,11.034402,NaN,11.034402,11.034402,11.034402,11.034402,11.034402
2,v1,validation,base,cum_fullhist+nostate,jga_turn,1.0,0.411581,NaN,0.411581,0.411581,0.411581,0.411581,0.411581
3,v1,validation,base,ops_partialhist+prev,jga_turn,1.0,17.141294,NaN,17.141294,17.141294,17.141294,17.141294,17.141294


In [15]:
all_results = (
    validation_results.loc[(validation_results["split"] == "test") & (validation_results["size"] == "base")]
    .drop(columns=["version", "split", "count"])
    .copy()
    .assign(
        state_repr=lambda df_: df_["exp"]
        .str.split("_", expand=True)[0]
        .map({"cum": "Cumulative", "ops": "Operations"}),
        context=lambda df_: df_["exp"]
        .str.split("_", expand=True)[1]
        .map(
            {
                "fullhist+nostate": "Full-history",
                "fullhist+prev": "Full-history + State",
                "nohist+prev": "State",
                "partialhist+prev": "4 Turns + State",
            }
        ),
        metric_name=lambda df_: df_["metric_name"].map(
            {
                "jga_turn_21": "2.1",
                "new_jga_turn_21": "2.1 (fix labels)",
                "jga_turn_22": "2.2",
                "new_jga_turn_22": "2.2 (fix labels)",
                "jga_turn_23": "2.3",
                "new_jga_turn_23": "2.3 (fix labels)",
                "jga_turn_24": "2.4",
                "new_jga_turn_24": "2.4 (fix labels)",
            }
        ),
    )[["state_repr", "context", "metric_name", "mean", "std", "min", "25%", "50%", "75%", "max"]]
    .rename(
        columns={
            "state_repr": "State representation",
            "context": "Context",
            "size": "Model size",
            "metric_name": "Dataset version",
        }
    )
    .set_index(["State representation", "Context", "Dataset version"])
    .round(2)
)

In [16]:
# full table
print(all_results.reset_index().iloc[:, 1:].to_latex(index=False))

\begin{tabular}{llrrrrrrr}
\toprule
        Context & Dataset version &  mean &  std &   min &   25\% &   50\% &   75\% &   max \\
\midrule
   Full-history &             NaN &  0.21 &  NaN &  0.21 &  0.21 &  0.21 &  0.21 &  0.21 \\
4 Turns + State &             NaN & 11.03 &  NaN & 11.03 & 11.03 & 11.03 & 11.03 & 11.03 \\
\bottomrule
\end{tabular}



---
### Run-time

In [18]:
tmp = pred_df["model"].str.split("_", expand=True)
pred_df["state_repr"] = tmp[1].str.strip()
pred_df["context"] = tmp[2].str.strip()
del tmp

In [20]:
pred_df["runtime_instance"] = pred_df["runtime"] / pred_df["batch_size"]

In [24]:
rt = pred_df.groupby(["state_repr", "context"])["runtime_instance"].agg(["median", "std"])

In [25]:
rt["relative"] = rt["median"] / rt["median"].min()

In [26]:
print(
    rt[["median", "relative"]]
    .reset_index()
    .assign(
        state_repr=lambda df_: df_["state_repr"].map({"cum": "Cumulative", "ops": "State operations"}),
        median=lambda df_: df_["median"] * 100,
    )
    .round(2)
    .to_latex(index=False)
)

\begin{tabular}{llrr}
\toprule
      state\_repr &          context &  median &  relative \\
\midrule
      Cumulative & fullhist+nostate &   77.16 &      3.82 \\
State operations & partialhist+prev &   20.18 &      1.00 \\
\bottomrule
\end{tabular}



In [63]:
schema = srsly.read_yaml("/data/processed/schema_guided_dialogue/schema.yaml")

In [104]:
pred_df.loc[(pred_df["dialogue_id"] == "10_00088") & (pred_df["split"] == "test"), "model"]

43342    sgd_ops_partialhist+prev_2022-12-27T15-24-56
43343    sgd_ops_partialhist+prev_2022-12-27T15-24-56
43346    sgd_ops_partialhist+prev_2022-12-27T15-24-56
43347    sgd_ops_partialhist+prev_2022-12-27T15-24-56
43350    sgd_ops_partialhist+prev_2022-12-27T15-24-56
                             ...                     
43571    sgd_cum_fullhist+nostate_2022-12-27T15-28-10
43574    sgd_cum_fullhist+nostate_2022-12-27T15-28-10
43575    sgd_cum_fullhist+nostate_2022-12-27T15-28-10
43578    sgd_cum_fullhist+nostate_2022-12-27T15-28-10
43579    sgd_cum_fullhist+nostate_2022-12-27T15-28-10
Name: model, Length: 120, dtype: object

In [110]:
pred_df.loc[
    (pred_df["jga_turn"] == True) & (~pred_df["states"].isna()),
    ["split", "dialogue_id", "turn_id", "model", "epoch", "input_text", "states_pred", "jga_turn"],
].set_index(["split", "dialogue_id", "turn_id", "epoch", "model"]).unstack(-1).dropna().shape

# .sort_values(["split", "dialogue_id", "turn_id", "model"])

(972, 6)

In [135]:
cum = pred_df.loc[
    pred_df["model"].str.contains("cum"),
    ["split", "dialogue_id", "turn_id", "epoch", "jga_turn", "input_text", "states_pred", "states"],
]
ops = pred_df.loc[
    ~pred_df["model"].str.contains("cum"),
    ["split", "dialogue_id", "turn_id", "epoch", "jga_turn", "input_text", "states_pred", "states"],
]

In [119]:
a = pd.merge(cum, ops, on=["split", "dialogue_id", "turn_id", "epoch"], suffixes=["_cum", "_ops"])

In [120]:
a.loc[a["jga_turn_ops"] == True]

,split,dialogue_id,turn_id,epoch,jga_turn_cum,input_text_cum,states_pred_cum,states,jga_turn_ops,input_text_ops,states_pred_ops
591,validation,10_00002,0,0,False,"generate full state: dialogue: system: none user: Hi, could you help me find some nice movies to watch please? <sep> history: system: 1695 Queen Street East. user: Thanks, that's all.; system: I've made your reservation. user: Where is it located?; system: Please confirm 1 ticket on March 12th. user: Great.; system: Please confirm 4 tickets for Mac Demarco in Toronto on March 12th. user: No, just 1 ticket.; system: Do you want to buy tickets? user: Yes, four tickets please.; system: Mac Demarco at Woodbine Park on March 12th. user: Sounds good.; system: 1695 Queen Street East at 5 pm. user: Anything else?; system: Mac Demarco at Woodbine Park is next Monday. user: Where is it located? When does it start?; system: 1695 Queen Street East at 5:30 pm. user: Sounds great. What are the dates?; system: I found 1 event. How about Mac Demarco at Woodbine Park? user: What's its location? When does it start?; system: I found Allan Rayman at August Hall. user: Anything in Toronto, Ontario?; system: What city? What are you looking for? user: Concert in San Fran.; system: none user: Please help me search for events. <sep> state:","{'events_2-city': 'toronto, ontario', 'events_2-date': 'march 12th', 'events_2-event_name': 'mac demarco', 'events_2-event_type': 'music', 'events_2-number_of_tickets': '1', 'movies_1-location': 'toronto, ontario'}","{'movies_3-cast': ['lori pelenise tuisano'], 'movies_3-directed_by': ['david leitch']}",True,"generate update operations: dialogue: system: none user: Hi, could you help me find some nice movies to watch please? <sep> previous dialogue states: none <sep> history: system: 1695 Queen Street East. user: Thanks, that's all.; system: I've made your reservation. user: Where is it located?; system: Please confirm 1 ticket on March 12th. user: Great.; system: Please confirm 4 tickets for Mac Demarco in Toronto on March 12th. user: No, just 1 ticket. <sep> operations:",None
593,validation,10_00002,0,0,False,"generate full state: dialogue: system: none user: Hi, could you help me find some nice movies to watch please? <sep> history: system: 1695 Queen Street East. user: Thanks, that's all.; system: I've made your reservation. user: Where is it located?; system: Please confirm 1 ticket on March 12th. user: Great.; system: Please confirm 4 tickets for Mac Demarco in Toronto on March 12th. user: No, just 1 ticket.; system: Do you want to buy tickets? user: Yes, four tickets please.; system: Mac Demarco at Woodbine Park on March 12th. user: Sounds good.; system: 1695 Queen Street East at 5 pm. user: Anything else?; system: Mac Demarco at Woodbine Park is next Monday. user: Where is it located? When does it start?; system: 1695 Queen Street East at 5:30 pm. user: Sounds great. What are the dates?; system: I found 1 event. How about Mac Demarco at Woodbine Park? user: What's its location? When does it start?; system: I found Allan Rayman at August Hall. user: Anything in Toronto, Ontario?; system: What city? What are you looking for? user: Concert in San Fran.; system: none user: Please help me search for events. <sep> state:","{'events_2-city': 'toronto, ontario', 'events_2-date': 'march 12th', 'events_2-event_name': 'mac demarco', 'events_2-event_type': 'music', 'events_2-number_of_tickets': '1', 'movies_1-location': 'toronto, ontario'}",None,True,"generate update operations: dialogue: system: none user: Hi, could you help me find some nice movies to watch please? <sep> previous dialogue states: none <sep> history: system: 1695 Queen Street East. user: Thanks, that's all.; system: I've made your reservation. user: Where is it located?; system: Please confirm 1 ticket on March 12th. user: Great.; system: Please confirm 4 tickets for Mac Demarco in Toronto on March 12th. user: No, just 1 ticket. <sep> operations:",None
599,validation,10_00002,0,2,False,"gen

In [127]:
a = ops.loc[(ops["jga_turn"] == True) & (~ops["states_pred"].isna())].assign(
    slots=lambda df_: df_["states_pred"].map(extract_slots),
    seen=lambda df_: df_["slots"].map(lambda ex: all(i in schema["train"] for i in ex)),
)

In [133]:
ops.columns

Index(['split', 'dialogue_id', 'turn_id', 'epoch', 'jga_turn', 'input_text',
       'states_pred'],
      dtype='object')

In [136]:
ops = ops.assign(
    slots=lambda df_: df_["states"].map(extract_slots),
    seen=lambda df_: df_["slots"].map(lambda ex: all(i in schema["train"] for i in ex)),
)

In [140]:
ops

,split,dialogue_id,turn_id,epoch,jga_turn,input_text,states_pred,states,slots,seen
0,validation,10_00000,0,0,False,"generate update operations: dialogue: system: none user: I wish to search a movie to watch online. Drama movie will be great. Search something directed by Likarion Wainaina. Search something iwth Stycie Waweru in it. <sep> previous dialogue states: none <sep> history: system: Will there be anything else? user: Nhat will be all. Thank you very much. You've been very helpful.; system: I have successfully made your reservation. user: Thank you very much. You've been very helpful.; system: Let me see if I got everything correctly: You'd like 4 tickets for the Timbers Vs Earthquakes event taking place on March 11th, in Portland. user: Yes, that's correct.; system: Please let me know how many tickets I should get. user: I'd like four tickets, please. <sep> operations:","{'media_1-directed_by': 'likarion wainaina', 'media_1-genre': 'drama'}","{'movies_3-directed_by': ['josephine decker'], 'movies_3-genre': ['suspense']}","[movies_3-directed_by, movies_3-genre]",False
1,validation,10_00000,0,0,False,"generate update operations: dialogue: system: none user: I wish to search a movie to watch online. Drama movie will be great. Search something directed by Likarion Wainaina. Search something iwth Stycie Waweru in it. <sep> previous dialogue states: none <sep> history: system: Will there be anything else? user: Nhat will be all. Thank you very much. You've been very helpful.; system: I have successfully made your reservation. user: Thank you very much. You've been very helpful.; system: Let me see if I got everything correctly: You'd like 4 tickets for the Timbers Vs Earthquakes event taking place on March 11th, in Portland. user: Yes, that's correct.; system: Please let me know how many tickets I should get. user: I'd like four tickets, please. <sep> operations:","{'media_1-directed_by': 'likarion wainaina', 'media_1-genre': 'drama'}","{'media_2-actors': ['stycie waweru'], 'media_2-director': ['likarion wainaina'], 'media_2-genre': ['drama']}","[media_2-actors, media_2-director, media_2-genre]",False
2,test,10_00000,0,0,False,generate update operations: dialogue: system: none user: I want to watch a suspense movie directed by Josephine Decker. <sep> previous dialogue states: none <sep> history: system: Do you need any other help? user: No. Thank you so much.; system: The wind speed is 12 miles per hour and the humidity is around 17 % user: Okay. Thank you.; system: The temperature is 82 degree Fahrenheit and only 7 % chance of rain. user: What about the windy there? How much humidity will be?; system: In which city shall I search in? user: Search in Palo Alto <sep> operations:,"{'media_1-directed_by': 'josephine decker', 'media_1-genre': 'suspense'}","{'movies_3-directed_by': ['josephine decker'], 'movies_3-genre': ['suspense']}","[movies_3-directed_by, movies_3-genre]",False
3,test,10_00000,0,0,False,generate update operations: dialogue: system: none user: I want to watch a suspense movie directed by Josephine Decker. <sep> previous dialogue states: none <sep> history: system: Do you need any other help? user: No. Thank you so much.; system: The wind speed is 12 miles per hour and the humidity is around 17 % user: Okay. Thank you.; system: The temperature is 82 degree Fahrenheit and only 7 % chance of rain. user: What about the windy there? How much humidity will be?; system: In which city shall I search in? user: Search in Palo Alto <sep> operations:,"{'media_1-directed_by': 'josephine decker', 'media_1-genre': 'suspense'}","{'media_2-actors': ['stycie waweru'], 'media_2-director': ['likarion wainaina'], 'media_2-genre': ['drama']}","[media_2-actors, media_2-director, media_2-genre]",False
4,validation,10_00000,0,2,False,"generate update operations: dialogue: system: none user: I wish to search a movie to watch online. Drama movie will be great. Search something directed by Likarion Wainaina. Search something iwth Stycie W